## ひろしまQuest2020#stayhome【球種予測部門】
#### all_merge_19
- コースの種類のみ

#### all_merge_18
- コースの種類、1球前のボール、ファウル数

#### all_merge_17
- イニング時間、打席時間、平均投球間隔(差)、サヨナラ、延長戦

#### all_merge_16
- 球団ごとの年棒ランク
- 選手IDごとの2017年のコース実績

#### all_merge_15
- 左右で球種/コースを別に集計

#### all_merge_14
- 左右で球種を別に集計

#### all_merge_13
- 前の投球からの時間差

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

### 管理番号

In [2]:
model_No = 19

### sub-modelを使用するかどうか

In [3]:
use_sub_model = False

In [4]:
ALL_MERGE = 'intermediate/all_merge_{}.f'.format(model_No)
SUBMIT = 'submit/ball_{}.csv'.format(model_No)

if use_sub_model:
    ALL_MERGE = 'intermediate/all_merge_{}_sub.f'.format(model_No)
    SUBMIT = 'submit/ball_{}_sub.csv'.format(model_No)

OUT_SUBMODEL = 'intermediate/ball_predict_{}.f'.format(model_No)

In [5]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 190)

In [6]:
if use_sub_model:
    all_pitch.drop(columns=[
        'predict_straight',
        'predict_curve',
        'predict_slider',
        'predict_shoot',
        'predict_fork',
        'predict_changeup',
        'predict_sinker',
        'predict_cutball'
    ], inplace=True)

In [7]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 190)


,No,ball,course,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,bc_right_ball,first,second,third,base_cnt,top_bot,role,pos_pit,date_from_opening,elapsed_min,min_diff,total_inning,elapsed_from_inning,ballnum_from_inning,...,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,rank_year_dif_p-b,rank_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,rank_year_dif_p-c,rank_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,rank_year_dif_b-c,rank_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.237468,0.325101,0.209547,0.145227,0.092978,0.027829,0.113620,0.135883,0.221489,0.204959,0.174254,0.263415,0.242045,0.294634,0.157681,0.115370,0.190270,0,0,0,0,0,1,0,0,0.000000,NaN,0,0.000000,1,...,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
1,1,0.0,8.0,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.210923,0.247198,0.177502,0.320359,0.147952,0.039943,0.116772,0.106283,0.230235,0.157397,0.138195,0.367890,0.184621,0.355720,0.111331,0.105923,0.242405,0,0,0,0,0,1,0,0,0.200000,0.200000,0,0.200000,2,...,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
2,2,0.0,5.0,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,1.0,0.138758,0.130040,0.092263,0.419906,0.117327,0.038140,0.046858,0.076007,0.284249,0.102015,0.095055,0.442674,0.118315,0.488095,0.076190,0.078571,0.238828,0,0,0,0,0,1,0,0,0.383333,0.183333,0,0.383333,3,...,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
3,3,0.0,12.0,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.0,0.0,0.213866,0.474931,0.139964,0.088519,0.038543,0.007023,0.138168,0.151308,0.196937,0.206965,0.157643,0.287147,0.105057,0.153718,0.164756,0.246104,0.330367,0,0,0,0,0,1,0,0,1.016667,0.633333,0,1.016667,4,...,320.0,154880.0,22.839506,268.0,81.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,0.325758,-253.0,3.191314,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,2040,18.0,12,-930.0,134880.0,-5.284091,192.0,-3.356681,0,0,0,1
4,4,0.0,8.0,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.0,0.0,0.215241,0.420099,0.145723,0.196762,0.051566,0.009680,0.118444,0.113468,0.206297,0.162901,0.127988,0.389347,0.084346,0.162167,0.120564,0.199445,0.433477,0,0,0,0,0,1,0,0,1.200000,0.183333,0,1.200000,5,...,320.0,154880.0,22.839506,268.0,81.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,0.325758,-253.0,3.191314,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,2040,18.0,12,-930.0,134880.0,-5.284091,192.0,-3.356681,0,0,0,1


### train
- 行数: 257117

In [8]:
train = all_pitch.dropna(subset=['course'])
train.shape

(257117, 190)

### test
- 行数: 521650

In [9]:
test = all_pitch[all_pitch['course'].isnull()]
test.shape

(521650, 190)

In [10]:
del all_pitch
gc.collect()

60

In [11]:
train_d = train.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

In [12]:
test_d = test.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

## Dataset作成

In [13]:
lgb_train = lgb.Dataset(train_d, train['ball'])

### LGBM学習
- multiclass : クラス数=8
- tuning結果のパラメータを使用
- 'learning_rate' : 0.1

In [14]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 8,
        'seed' : 0,
        'learning_rate' : 0.1,
        'lambda_l1': 6.9923570049658075, 
        'lambda_l2': 0.002378623984798833, 
        'num_leaves': 18, 
        'feature_fraction': 0.45199999999999996, 
        'bagging_fraction': 0.9799724836460725, 
        'bagging_freq': 4, 
        'min_child_samples': 20
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['multi_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

[100]	cv_agg's multi_logloss: 1.14083 + 0.00251911
[200]	cv_agg's multi_logloss: 1.11913 + 0.00262731
[300]	cv_agg's multi_logloss: 1.11277 + 0.00262988
[400]	cv_agg's multi_logloss: 1.10877 + 0.00259839
[500]	cv_agg's multi_logloss: 1.1062 + 0.00267275
[600]	cv_agg's multi_logloss: 1.10427 + 0.00274456
[700]	cv_agg's multi_logloss: 1.10279 + 0.00283276
[800]	cv_agg's multi_logloss: 1.10189 + 0.00297764
[900]	cv_agg's multi_logloss: 1.10119 + 0.00300222
[1000]	cv_agg's multi_logloss: 1.10085 + 0.00303837
[1100]	cv_agg's multi_logloss: 1.10049 + 0.00323504
[1200]	cv_agg's multi_logloss: 1.10047 + 0.00330494
Best num_boost_round: 1141
Best CV score: 1.1004105063817775
CPU times: user 1h 42min 37s, sys: 19.8 s, total: 1h 42min 56s
Wall time: 12min 53s


In [15]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [16]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 32min 20s, sys: 6.08 s, total: 32min 26s
Wall time: 4min 3s


### Feature Importance

In [17]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
115,mid_str,1525
174,bmi_dif_p-c,1536
120,center_str,1582
162,salary_year_dif_p-b,1588
141,bat_game_cnt,1597
60,point_diff,1656
102,curve,1709
105,fork,1735
180,rank_year_dif_b-c,1757
139,bmi_bat,1762


In [18]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 6min 56s, sys: 800 ms, total: 6min 57s
Wall time: 53.6 s


## 結果出力

In [19]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.540643,0.042029,0.137291,0.204651,0.003142,0.000014,0.000020,0.072211
1,1,0.547003,0.109775,0.123600,0.180890,0.006437,0.000016,0.000010,0.032270
2,2,0.302447,0.098551,0.169209,0.114406,0.208372,0.000105,0.000029,0.106881
3,3,0.382849,0.135841,0.208349,0.191833,0.023578,0.000037,0.000022,0.057491
4,4,0.180641,0.050297,0.415122,0.229313,0.013071,0.000057,0.000029,0.111470


In [20]:
submit.to_csv(SUBMIT, header=False, index=False)

### コース予測で使用

In [21]:
%%time
if not use_sub_model:
    train_predict = lgb_model.predict(train_d, num_iteration = lgb_model.best_iteration)

CPU times: user 3min 24s, sys: 316 ms, total: 3min 24s
Wall time: 26.2 s


In [22]:
if not use_sub_model:
    df_train_predict = pd.DataFrame(train_predict).reset_index()
    submodel = pd.concat([df_train_predict, submit], ignore_index=True)
    submodel.drop(columns=['index'], inplace=True)
    submodel.rename(columns={
        0: 'predict_straight',
        1: 'predict_curve',
        2: 'predict_slider',
        3: 'predict_shoot',
        4: 'predict_fork',
        5: 'predict_changeup',
        6: 'predict_sinker',
        7: 'predict_cutball'
    }, inplace=True)
    print(submodel.shape)
    submodel.head()

(778767, 8)


In [23]:
if not use_sub_model:
    submodel.to_feather(OUT_SUBMODEL)

### SIGNATE

In [24]:
'signate submit --competition-id=275 ./{} --note feat={}_cv={}'.format(SUBMIT, column_cnt, best_cv_score)

'signate submit --competition-id=275 ./submit/ball_19.csv --note feat=190_cv=1.1004105063817775'

### GCS

In [25]:
'gsutil cp ./{} gs://yasunobu_baseball/submit/'.format(SUBMIT)

'gsutil cp ./submit/ball_19.csv gs://yasunobu_baseball/submit/'

### 評価結果
#### 19 (190) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1141
- Best CV score: 1.1004105063817775
- Wall time: 12min 53s
- 評価結果  : 1.3974324

#### 18_sub (230) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1204
- Best CV score: 1.1004215512272495
- Wall time: 15min 34s
- 評価結果  : 1.3953962 ★

#### 18 (217) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1102
- Best CV score: 1.1004234715781385
- Wall time: 13min 42s
- 評価結果  : 1.3954167

#### 17 (191) 
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1100
- Best CV score: 1.1034107856339535
- Wall time: 26min 44s
- 評価結果  : 1.3963057

#### 16 (181) ... 選手IDごとの2017年のコース実績
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1178
- Best CV score: 1.1021930539837317
- Wall time: 12min 35s
- 評価結果  : 1.3993836

#### 15 (153) ... 左右で球種/コースを別に集計
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1189
- Best CV score: 1.1013869435441306
- Wall time: 11min 9s
- 評価結果  : 未提出

#### 14 (153) ... 左右で球種を別に集計
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1240
- Best CV score: 1.1021952294615331
- Wall time: 11min 34s
- 評価結果  : 1.3981900

#### 13 (157) ... 前の投球からの時間差を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1416
- Best CV score: 1.1062463941695753
- Wall time: 13min 10s
- 評価結果  : 1.4012184

#### 12 (156) ... コースの集計を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1353
- Best CV score: 1.106942086205103
- Wall time: 27min 41s
- 評価結果  : 1.4016077

#### 11 (143) ... ボールカウントの集計を追加
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1209
- Best CV score: 1.1075153721728936
- Wall time: 23min 19s
- 評価結果  : 未提出

#### 10_tune (135) ... 再チューニング
- 'num_leaves' : 15,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1260
- Best CV score: 1.1118897437428692
- Wall time: 8min 40s
- 評価結果  : 1.4060397

#### 10s (135) ... trainを75%サンプリング
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1412
- Best CV score: 1.1190287120710383
- Wall time: 12min 5s
- 評価結果  : 1.4310869

#### 10 (135) ... 5相当に戻した
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1112
- Best CV score: 1.111265000190272
- Wall time: 9min 30s
- 評価結果  : 1.4008712

#### 9 (136) ... 5相当に戻した=>NG
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1106
- Best CV score: 1.1113177029272225
- Wall time: 25min 44s (local)
- 評価結果  : 1.5133521

#### 8 (157)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1100
- Best CV score: 1.1093728644236824
- Wall time: 10min 4s
- 評価結果  : 1.4925832

#### 7 (235)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1204
- Best CV score: 1.108590082437519
- Wall time: 12min 56s
- 評価結果  : 1.4946937

#### 6 (207)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1135
- Best CV score: 1.1086837935965734
- Wall time: 11min 54s
- 評価結果  : 1.3999194

#### 5-tune (参考)
- 'num_leaves' : 18,
- 'learning_rate' : 0.05,
- Best num_boost_round: 2395
- Best CV score: 1.1103399384119979
- Wall time: 18min 29s (n1-standard-8)

#### 5-tune (135)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- CV score: 1.1112402566616455
- 評価結果  : 1.3991780
- CV Wall time: 27min 48s (local) -> 18min 24s (n1-standard-4) -> 9min 3s (n1-standard-8)

#### 5 (135)
- 'num_leaves' : 31,
- 'learning_rate' : 0.1,
- CV score: 1.1161825304353852
- 評価結果  : 1.4252286

#### 4 (128)
- CV score: 1.1158111693845418
- 評価結果  : 1.4299265

#### 3 (97)
- CV score: 1.1172290313076036
- 評価結果  : 1.4335806